In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib notebook

from ipywidgets import interact, interactive, fixed, interact_manual

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML, Audio
import matplotlib.pyplot as plt
from scipy import signal # convolution, 

from ient_nb.ient_plots import *
from ient_nb.ient_signals import *

<div>
    <img src="ient_nb/figures/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

# Augendiagramm

In [ ]:
def convolution(s, h):
    # Convolve s and h numerically
    g = signal.convolve(s, h, mode='full')*deltat; g = g[0:len(s)];
    return g
raisedCosine = lambda t: rect(t)*np.cos(np.pi*t)**2
raisedCosine = lambda t: 0.5*si(np.pi*t)+0.25*si(np.pi*(t+1))+0.25*si(np.pi*(t-1))

fs = 1000 # Abtastrate
La = 100 # Anzahl an Symbolen

T=1 # Breite des Trägersignals in Sekunden
Tmax = La*T # Gesamtlänge des Sendesignals

# Achsen
(t,deltat) = np.linspace(0, Tmax, Tmax*fs, retstep=True) # Zeitachse

In [ ]:
a = np.random.randint(0, 2, size=La) # Nachricht
a[0:10]

In [ ]:
## Sender

# Trägersignal
s = lambda t: rect(t/T-0.5) 
#s = lambda t: raisedCosine(t/T*3-0.5)

# Sendesignal
m = np.zeros(len(t))
for l in range(len(a)):
    m = m + a[l]*s(t-l*T)

## Kanal
n = np.random.normal(0,1,len(t));
x = m+n
#x=m

## Empfänger
y = convolution(x, s(T-t))

# Plot
fig,axs = plt.subplots(1,2); 
ax = axs[0]; ax.plot(t/T, m, 'rwth');
ax.set_xlabel(r'$\rightarrow t/T$'); ax.set_ylabel(r'$m(t)$'); 
ax.set_xlim([0,20]); ax.set_ylim([0,1.09]); ient_axis(ax); 
ax = axs[1]; ax.plot(t/T, y, 'rwth');
ax.set_xlabel(r'$\rightarrow t/T$'); ax.set_ylabel(r'$y(t)$'); 
ax.set_xlim([0,20]); ax.set_ylim([0,1.09]); ient_axis(ax); 

## Augendiagramm

In [ ]:
def eyediagram(x, L, shift=0):
    
    k_max = int((len(x) - shift)/L)-1
    x = x[0:k_max*L]
    
    E = np.reshape(x, (-1,L)).T
    t = np.linspace(-T,T,L)
    return (E,t)

E,tE = eyediagram(y,int(T*fs*2))

# Plot
fix,ax = plt.subplots(1,1); ax.plot(tE, E, 'RWTH');
ax.set_xlabel(r'$\rightarrow t/T$', bbox=ient_wbbox); ax.set_ylabel(r'$\uparrow y(t)$', bbox=ient_wbbox);
ax.set_xlim([-1,1.19]); ax.set_ylim([0,1.09]); ax.grid(); ient_axis(ax)

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources) (OER). Feel free to use the notebook for your own purposes. The code is licensed under the [MIT license](https://opensource.org/licenses/MIT). 

Please attribute the work as follows: 
*Christian Rohlfing, Übungsbeispiele zur Vorlesung "Informationsübertragung"*, gehalten von Jens-Rainer Ohm, 2019, Institut für Nachrichtentechnik, RWTH Aachen University.